In [67]:
import json, requests, time, datetime, math
from pprint import pprint
import numpy as np
import pandas as pd
from scipy.stats import norm
from pandas_datareader import data as pdr
from functools import reduce
import constants, web3_service, chain_data_service

# Used to find portfolio weights for an array of balances
def getWeights(balances):
    total = 0
    weights = []
    for balance in balances:
        total += balance['balance']
    for balance in balances:
        percentage = balance['balance']/total
        weights.append(percentage)
    return weights

# Used to find historical USD values for stablecoins
def getCryptoCompareReturns(token):
    result = requests.get(f'https://min-api.cryptocompare.com/data/v2/histoday?fsym={token}&tsym=USD&limit=720')
    json = result.json()
    df = pd.DataFrame([object for object in json['Data']['Data']])
    df.insert(0, 'Date', pd.to_datetime(df['time'],unit='s'))
    df.drop(['high', 'open', 'low', 'volumefrom', 'volumeto', 'conversionType', 'conversionSymbol', 'time'], axis=1, inplace=True)
    df.set_index('Date', inplace=True)
    returns = df.copy().pct_change().fillna(value=0, axis=0).rename(columns={'close': f'daily_returns_{token}'})
    return returns
    
# TODO - Fix Dates
# Used to find historical USD values for all coins but stablecoins      
def getReturns(tokens): 
    first_date = '2017-10-06'
    last_date = '2019-09-25'
    df_list = []
    for token in tokens:
        if (token['token'][0] == 'i' and token['token'][1] == 'w'):
            token = token['token'][2:].upper()
        elif (token['token'][0] == 'c' or token['token'][0] ==  'w' or token['token'][0] == 'i'):
            token = token['token'][1:].upper()
        else:
            token = token['token'].upper()
        if (token == 'DAI' or token == 'USDC' or token == 'MKR' or token == 'TUSD'):
            ticker_returns = getCryptoCompareReturns(token)
            df_list.append(ticker_returns)
        else:
            ticker = f'{token}-USD'
            ticker_close = pdr.get_data_yahoo(ticker, first_date, last_date)[['Close']]
            ticker_returns = ticker_close.copy().pct_change().fillna(value=0, axis=0).rename(columns={'Close': f'daily_returns_{token}'})
            df_list.append(ticker_returns)
    df = reduce(lambda x, y: pd.merge(x, y, on = 'Date'), df_list)
    return df

def value_at_risk(returns, weights, alpha=0.95, lookback_days=520):
    # Multiply asset returns by weights to get one weighted portfolio return
    portfolio_returns = returns.iloc[-lookback_days:].dot(weights)
    # Compute the correct percentile loss and multiply by value invested
    return np.percentile(portfolio_returns, 100 * (1-alpha))

def cvar(returns, weights, alpha=0.95, lookback_days=520):
    # Call out to our existing function
    var = value_at_risk(returns, weights, alpha, lookback_days=lookback_days)
    returns = returns.fillna(0.0)
    portfolio_returns = returns.iloc[-lookback_days:].dot(weights)
    # Get back to a return rather than an absolute loss
    var_pct_loss = var / 1
    return np.nanmean(portfolio_returns[portfolio_returns < var_pct_loss])

def generate_cvar_from_balances(balances):
    weights = getWeights(balances)
    returns = getReturns(balances)
    returns = returns.fillna(0.0)
    portfolio_returns = returns.fillna(0.0).iloc[-520:].dot(weights)
    portfolio_cvar = cvar(returns, weights, 0.99, 520)
    return portfolio_cvar

def EMACalc(m_array, m_range):
    k = 2/(m_range + 1)
    # first item is just the same as the first item in the input
    ema_array = [m_array[0]]
    # for the rest of the items, they are computed with the previous one
    i = 1
    while i < m_range:
        ema_array.append(m_array[i] * k + ema_array[i - 1] * (1 - k))
        i += 1
    return ema_array[len(ema_array) - 1]

# Normalize value from a list of objects with predefined shape
def normalize_data(val, list):
    max_value = max(list, key=lambda x: x['value'])['value']
    min_value = min(list, key=lambda x: x['value'])['value']
    return (val - min_value) / (max_value - min_value)

# Get Collateralization data, using either Alethio API or Nuo API
def get_collateralizations():
    collateralization_list = []
    start_date = datetime.datetime.now() + datetime.timedelta(-30)
    start_date_unix = round(time.mktime(start_date.timetuple()))
    data = requests.get(f'https://api.defi.alethio.tech/v0/stats?after={start_date_unix}&granularity=24&protocols=compound,bzx,dydx').json()
    for market in data['data']:
        collateral_array = np.asarray([item[1] for item in market['collateral_ratio']], dtype=np.float64)
        collateral_ema = EMACalc(collateral_array, 30)
        market_name = f"{market['protocol']}-{market['asset']}"
        collateralization_list.append({ 'market': market_name, 'value': math.log(collateral_ema)  })
    nuo_data = requests.get('https://api.nuoscan.io/overview').json()
    # Don't have time series data for collateralization for Nuo yet, so using single point in time
    # Ignoring SNX, KNC, and MKR markets for now
    outstanding_debt = sum(m['active_loan_amount_sum_primary'] for m in nuo_data['data']['reserves'] if m['currency']['short_name'] not in ['SNX', 'KNC', 'MKR'])
    total_assets = sum(m['total_balance_primary'] for m in nuo_data['data']['reserves'] if m['currency']['short_name'] not in ['SNX', 'KNC', 'MKR'])
    collateral_ratio = total_assets / outstanding_debt
    for market in nuo_data['data']['reserves']:
        if market['currency']['short_name'] not in ['SNX', 'KNC', 'MKR']:
            market_name = f"nuo-{market['currency']['short_name'].lower()}"
            collateralization_list.append({ 'market': market_name, 'value': math.log(collateral_ratio)  })
    return collateralization_list

# Get Liquidity data, using either Alethio API or Nuo API
def get_liquidities():
    liquidity_list = []
    for balance in fulcrum_balances:
        market_name = f'bzx-{balance["token"]}'
        liquidity_list.append({ 'market': market_name, 'value': math.log(balance['balance']) })
    for balance in dydx_balances:
        market_name = f'dydx-{balance["token"]}'
        liquidity_list.append({ 'market': market_name, 'value': math.log(balance['balance']) })
    for balance in compound_balances:
        market_name = f'compound-{balance["token"]}'
        liquidity_list.append({ 'market': market_name, 'value': math.log(balance['balance']) })
    for balance in nuo_balances:
        market_name = f"nuo-{balance['token']}"
        liquidity_list.append({ 'market': market_name, 'value': math.log(balance['balance']) })
    return liquidity_list

# TODO - make this function cleaner
# Score Calculation function
def calculate_score(protocol, token):
    if protocol == 'dydx':
        protocol_values = constants.dydx_values
    elif protocol == 'compound':
        protocol_values = constants.compound_values
    elif protocol == 'bzx':
        protocol_values = constants.fulcrum_values
    else:
        protocol_values = constants.nuo_values
    liquidity_array = get_liquidities()
    liquidity_value = next((x for x in liquidity_array if x['market'] == f'{protocol}-{token}'), None)['value']
    normalized_liquidity_value = normalize_data(liquidity_value, liquidity_array)
    collateralization_array = get_collateralizations()
#     pprint(collateralization_array)
    collateralization_value = next((x for x in collateralization_array if x['market'] == f'{protocol}-{token}'), None)['value']
    normalized_collateralization_value = normalize_data(collateralization_value, collateralization_array)
    # Calculate score
    pprint(normalized_collateralization_value)
    pprint(normalized_liquidity_value)
    weights = constants.weights
    score = weights['auditedCode'] * protocol_values['isCodeAudited'] + weights['allCodeOSS'] * protocol_values['isCodeOpenSource'] + weights['formalVer'] * protocol_values['isCodeFormallyVerified'] + weights['hasBugBounty'] * protocol_values['hasBugBounty'] + weights['cVaR'] * protocol_values['cvar'] + weights['poolCollateralization'] * normalized_collateralization_value + weights['poolLiquidity'] * normalized_liquidity_value
    score = round(score, 2) * 10
    return score

In [68]:
# Pulling and calculating Compound data
compound_balances = list(map(chain_data_service.fetch_data_for_compound_market, constants.compoundContractInfo))
compound_portfolio_cvar = generate_cvar_from_balances(compound_balances)
# add instead of subtract here because cvar from this function is negative
constants.compound_values['cvar'] = 1 + compound_portfolio_cvar
pprint(constants.compound_values)

{'cvar': 0.8861251975146153,
 'hasBugBounty': 1,
 'insuranceRisk': 0,
 'isCodeAudited': 1,
 'isCodeFormallyVerified': 1,
 'isCodeOpenSource': 1}


In [57]:
# Pulling and calculating dYdX data
dydx_balances = list(chain_data_service.fetch_data_for_dydx_market(x) for x in [0, 1, 2])
dydx_portfolio_cvar = generate_cvar_from_balances(dydx_balances)
# add instead of subtract here because cvar from this function is negative
constants.dydx_values['cvar'] = 1 + dydx_portfolio_cvar
pprint(constants.dydx_values)

{'cvar': 0.8659315839731011,
 'hasBugBounty': 1,
 'insuranceRisk': 0,
 'isCodeAudited': 1,
 'isCodeFormallyVerified': 0,
 'isCodeOpenSource': 1}


In [58]:
# Pulling and calculating Fulcrum data
fulcrum_balances = list(map(chain_data_service.fetch_data_for_fulcrum_market, constants.fulcrumContractInfo))
fulcrum_portfolio_cvar = generate_cvar_from_balances(fulcrum_balances)
# add instead of subtract here because cvar from this function is negative
constants.fulcrum_values['cvar'] = 1 + fulcrum_portfolio_cvar
pprint(constants.fulcrum_values)

{'cvar': 0.9316391593643878,
 'hasBugBounty': 0,
 'insuranceRisk': 0,
 'isCodeAudited': 1,
 'isCodeFormallyVerified': 0,
 'isCodeOpenSource': 1}


In [59]:
# Pulling and calculating Nuo data
nuo_balances = list(map(chain_data_service.fetch_data_for_nuo_market, constants.nuoContractInfo))
nuo_portfolio_cvar = generate_cvar_from_balances(nuo_balances)
# add instead of subtract here because cvar from this function is negative
constants.nuo_values['cvar'] = 1 + nuo_portfolio_cvar
pprint(constants.nuo_values)

{'cvar': 0.9174366784383677,
 'hasBugBounty': 0,
 'insuranceRisk': 0,
 'isCodeAudited': 0,
 'isCodeFormallyVerified': 0,
 'isCodeOpenSource': 1}


In [69]:
calculate_score('compound', 'eth')

0.7035976966427006
1.0


7.9

In [70]:
calculate_score('compound', 'dai')

0.7035968751289499
0.8779502704503265


7.800000000000001

In [71]:
calculate_score('compound', 'bat')

0.7035968751289499
0.7155389799864126


7.7

In [72]:
calculate_score('compound', 'zrx')

0.7035976966427006
0.7418699432588205


7.7

In [73]:
calculate_score('compound', 'usdc')

0.7035976966427006
0.8897747005973563


7.800000000000001

In [74]:
calculate_score('compound', 'wbtc')

0.7035968751289499
0.674558496159999


7.6

In [75]:
calculate_score('compound', 'rep')

0.7035976966427006
0.8063648261424086


7.7

In [76]:
calculate_score('dydx', 'usdc')

0.09218760989139084
0.6755245637501309


6.2

In [77]:
calculate_score('dydx', 'dai')

0.09218760989139084
0.7587129501251869


6.3

In [78]:
calculate_score('dydx', 'eth')

0.09218760989139084
0.9037634618708084


6.4

In [79]:
calculate_score('bzx', 'eth')

0.0
0.5785326954845705


5.5

In [80]:
calculate_score('bzx', 'dai')

0.0
0.595747477417853


5.5

In [81]:
calculate_score('bzx', 'usdc')

0.0
0.5113020264754392


5.4

In [82]:
calculate_score('bzx', 'rep')

0.0
0.0


4.9

In [83]:
calculate_score('bzx', 'wbtc')

0.0
0.49977977474818447


5.4

In [84]:
calculate_score('bzx', 'link')

0.0
0.3905443850264408


5.300000000000001

In [85]:
calculate_score('bzx', 'knc')

0.0
0.14895675368132663


5.1

In [86]:
calculate_score('bzx', 'zrx')

0.0
0.19278891394496975


5.1

In [87]:
calculate_score('nuo', 'usdc')

1.0
0.7186224784794116


4.6000000000000005

In [88]:
calculate_score('nuo', 'eth')

1.0
0.6681999589897586


4.6000000000000005

In [89]:
calculate_score('nuo', 'dai')

1.0
0.6675568053550027


4.6000000000000005

In [90]:
calculate_score('nuo', 'wbtc')

1.0
0.5748221096256262


4.5

In [91]:
calculate_score('nuo', 'zrx')

1.0
0.43997969740548465


4.4

In [92]:
calculate_score('nuo', 'bat')

1.0
0.4913666764041936


4.4

In [93]:
calculate_score('nuo', 'link')

1.0
0.7326494530878305


4.699999999999999

In [94]:
calculate_score('nuo', 'tusd')

1.0
0.44987245275581983


4.4

In [95]:
calculate_score('nuo', 'rep')

1.0
0.3797798116229816


4.3